In [250]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [251]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

test_ids = test["PassengerId"]

data.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [252]:
def cleanup(dataset):
    dataset.drop(["PassengerId", "SibSp", "Ticket", "Name"], axis=1, inplace=True)
    return dataset

data = cleanup(data)
test = cleanup(test)

data.head(5)

def fix_cabin(x):
    if not pd.isnull(x):
        return str(x)[0]
    else:
        return x

data["Cabin"] = data["Cabin"].apply(fix_cabin)
test["Cabin"] = test["Cabin"].apply(fix_cabin)
data.head(5)

,Survived,Pclass,Sex,Age,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,0,7.2500,NaN,S
1,1,1,female,38.0,0,71.2833,C,C
2,1,3,female,26.0,0,7.9250,NaN,S
3,1,1,female,35.0,0,53.1000,C,S
4,0,3,male,35.0,0,8.0500,NaN,S


In [253]:
nancols = data.columns[data.isna().any()].tolist()
test_nancols = test.columns[test.isna().any()].tolist()

data["Age"].fillna(data["Age"].mean(axis=0), inplace=True)
data["Cabin"].fillna(data["Cabin"].value_counts().index[0], inplace=True)
data["Embarked"].fillna(data["Embarked"].value_counts().index[0], inplace=True)

test["Age"].fillna(test["Age"].mean(axis=0), inplace=True)
test["Fare"].fillna(test["Fare"].mean(axis=0), inplace=True)
test["Cabin"].fillna(test["Cabin"].value_counts().index[0], inplace=True)
test["Embarked"].fillna(test["Embarked"].value_counts().index[0], inplace=True)

data.isnull().values.any(), test.isnull().values.any()

(False, False)

In [254]:
nonnumerical_cols = ["Sex", "Cabin", "Embarked"]
encoder = LabelEncoder()

for col in nonnumerical_cols:
    data[col] = encoder.fit_transform(data[col])
    test[col] = encoder.transform(test[col])

data.head(5)

,Survived,Pclass,Sex,Age,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,0,7.2500,2,2
1,1,1,0,38.0,0,71.2833,2,0
2,1,3,0,26.0,0,7.9250,2,2
3,1,1,0,35.0,0,53.1000,2,2
4,0,3,1,35.0,0,8.0500,2,2


In [255]:
y = data["Survived"]
X = data.drop("Survived", axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
X_train

,Pclass,Sex,Age,Parch,Fare,Cabin,Embarked
834,3,1,18.0,0,8.3000,2,2
342,2,1,28.0,0,13.0000,2,2
583,1,1,36.0,0,40.1250,0,0
823,3,0,27.0,1,12.4750,4,2
396,3,0,31.0,0,7.8542,2,2
...,...,...,...,...,...,...,...
419,3,0,10.0,2,24.1500,2,2
66,2,0,29.0,0,10.5000,5,2
737,1,1,35.0,0,512.3292,1,0
785,3,1,25.0,0,7.2500,2,2


In [256]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

model = KNeighborsClassifier(n_neighbors=16)
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=16)

In [257]:
val_preds = model.predict(X_val)
print(accuracy_score(y_val, val_preds))
confusion_matrix(y_val, val_preds)

0.6666666666666666


array([[47, 12],
       [18, 13]], dtype=int64)

In [258]:
preds = model.predict(test)

In [259]:
df = pd.DataFrame({
    "PassengerId": test_ids.values,
    "Survived": preds,
})

In [260]:
df.to_csv("submission.csv", index=False)